In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [2]:
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0",
    "Accept": "*/*",
    "Connection": "close",
}

In [3]:
url="https://arxiv.org/list/cs/pastweek?skip=0&show=100"
page=requests.get(url=url,headers=header)
soup=BeautifulSoup(page.text,'html.parser')

In [ ]:
abstracts_soup=soup.select('dt span a[title="Abstract"]')
abstracts_url=["https://arxiv.org"+u['href'] for u in abstracts_soup]
abstract=[]
for i in range(100):
    index=abstracts_url[i]
    status=0
    while status!=200:
        try:
            html = requests.get(index, timeout=2)
            status=html.status_code
        except requests.Timeout as e:
            print("Retry.......")
        sleep(1)
    soups=BeautifulSoup(html.text,'html.parser')
    abstract.append(soups.select('meta[property="og:description"]')[0]['content'])
    print(f"Sucessful at {index}")
    sleep(1)

Retry.......
Sucessful at https://arxiv.org/abs/2403.00762
Sucessful at https://arxiv.org/abs/2403.00758
Retry.......
Sucessful at https://arxiv.org/abs/2403.00752
Sucessful at https://arxiv.org/abs/2403.00745
Sucessful at https://arxiv.org/abs/2403.00743
Sucessful at https://arxiv.org/abs/2403.00742
Retry.......
Sucessful at https://arxiv.org/abs/2403.00737
Retry.......
Sucessful at https://arxiv.org/abs/2403.00729
Sucessful at https://arxiv.org/abs/2403.00725
Sucessful at https://arxiv.org/abs/2403.00724
Sucessful at https://arxiv.org/abs/2403.00720
Sucessful at https://arxiv.org/abs/2403.00717
Sucessful at https://arxiv.org/abs/2403.00715
Sucessful at https://arxiv.org/abs/2403.00712
Retry.......
Sucessful at https://arxiv.org/abs/2403.00704
Sucessful at https://arxiv.org/abs/2403.00696
Retry.......
Sucessful at https://arxiv.org/abs/2403.00691
Sucessful at https://arxiv.org/abs/2403.00690
Retry.......
Sucessful at https://arxiv.org/abs/2403.00689
Sucessful at https://arxiv.org/abs/

In [ ]:
title=[]
Subjects=[]
Authors=[]
for article in soup.select('dd'):
    title.append(article.select_one('div[class="list-title mathjax"]').text.replace("\n","").replace("Title: ",""))
    Authors.append(article.select_one('div[class="list-authors"]').text.replace("\n","").replace("Authors:",""))
    Subjects.append(article.select_one('div span[class="primary-subject"]').text)

In [ ]:
df=pd.DataFrame({"title":title,"Subjects":Subjects,"Authors":Authors,"abstract":abstract})
df.to_csv("First_100_papers.csv",index=False)